# Affect for RAF_DBimport numpy as np

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import dask.dataframe as dd
from dask.multiprocessing import get
import cv2
from tqdm import tqdm

import matplotlib.pyplot as plt

tqdm.pandas()

In [2]:
RAF_DB_label = ["Surprise", "Fear", "Disgust", "Happiness", "Sadness", "Anger", "Neutral"]
Affect_label = ["surprise", "fear", "disgust", "happy", "sad", "anger", "neutral"]

In [8]:
label_path = "/run/media/viper/LSP/Dataset/Affect/labels.csv"
df = pd.read_csv(label_path, sep=",")
df.head()

,pth,label
0,anger/image0000006.jpg,surprise
1,anger/image0000007.jpg,anger
2,anger/image0000012.jpg,anger
3,anger/image0000035.jpg,fear
4,anger/image0000060.jpg,anger


In [9]:
df = df.drop(df[df["label"] == "contempt"].index)

In [10]:
df["file"] = 0

In [11]:
chunksize = 10
se_index = np.linspace(0, len(df), chunksize).astype(int)
for i in range(chunksize):
    X_affectnet = np.zeros((se_index[i+1]-se_index[i], 224, 224, 3))
    for j in tqdm(range(se_index[i], se_index[i+1])):
        img = cv2.imread("/run/media/viper/LSP/Dataset/Affect/" + df.iloc[i]["pth"])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        resized = cv2.resize(img, (224, 224), interpolation = cv2.INTER_AREA)
        X_affectnet[j-se_index[i],:,:,:] = resized
        df["file"].iloc[j] = i
    np.save("/run/media/viper/LSP/Dataset/IRP_data/Affect_7class/X_affectnet"+ str(i) + ".npy", X_affectnet, allow_pickle=True)

  0%|          | 0/3046 [00:00<?, ?it/s]<ipython-input-11-1d583e5752d4>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["file"].iloc[j] = i
100%|██████████| 3046/3046 [04:19<00:00, 11.75it/s]


IndexError: index 10 is out of bounds for axis 0 with size 10

In [13]:
df = df.drop(df[df["file"] == 0].index)

In [15]:
df.to_csv("/run/media/viper/LSP/Dataset/IRP_data/Affect_7class/Affect7.csv")